<hr style="margin-bottom: 50px;">
<center>
    <h1 style="margin-top: 0; margin-bottom: 0;">
        <b><u>Modélisation : SARIMAX</u></b>
    </h1>
</center>
<hr style="margin-top: 50px;">

# <b>1. Configuration du notebook

In [1]:
# Imports des modules du projet
import config, src

# Imports classiques
import numpy as np
import pandas as pd

# Imports de scikit-learn
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Imports de statsmodels
from statsmodels.tsa.arima.model import ARIMA

# Imports pour la visualisation
import plotly.graph_objects as go

/Users/lucascaio/Desktop/Fac/Master/M2/2. Bloc fondamental/Apprentissage sequentiel pour la prevision de series temporelles/appliances-energy-prediction/notebooks


In [2]:
# Chargement des données prétraitées
data = src.load_data(config.PROCESSED_DATA_FILE)

# Conversion des données au format datetime
data['Date'] = pd.to_datetime(data['Date'])

# Utilisation des dates comme index
data.set_index('Date', inplace=True)

Données chargées avec succès.


In [3]:
def borne_min(array, valeur_borne):
    return (array >= valeur_borne) * array + (array <= valeur_borne) * valeur_borne

borne_min(np.array([1, 0, -2, 5, 7]), 0.5)

array([1. , 0.5, 0.5, 5. , 7. ])

In [5]:
t = np.arange(len(data))

X = pd.DataFrame({"Temps": t,"Sin_day": np.sin((2*np.pi*(t))/144),"intercept": np.ones((len(data),))})

variables_explicatives_list = data.columns.drop('Energy')

for j in range(24):
    X[f'is_{j+1}th_hour'] = (data.index.hour == j).astype(int)
        
for j in range(7):    
    X[f'is_{j+1}th_day'] = (data.index.day == j).astype(int)


X[variables_explicatives_list] = data[variables_explicatives_list].copy().to_numpy()

y = data["Energy"].copy()

train_index = (data.index <= "2016-04-27 18:00:00").sum()


In [11]:
X.columns

Index(['Temps', 'Sin_day', 'intercept', 'is_1th_hour', 'is_2th_hour',
       'is_3th_hour', 'is_4th_hour', 'is_5th_hour', 'is_6th_hour',
       'is_7th_hour', 'is_8th_hour', 'is_9th_hour', 'is_10th_hour',
       'is_11th_hour', 'is_12th_hour', 'is_13th_hour', 'is_14th_hour',
       'is_15th_hour', 'is_16th_hour', 'is_17th_hour', 'is_18th_hour',
       'is_19th_hour', 'is_20th_hour', 'is_21th_hour', 'is_22th_hour',
       'is_23th_hour', 'is_24th_hour', 'is_1th_day', 'is_2th_day',
       'is_3th_day', 'is_4th_day', 'is_5th_day', 'is_6th_day', 'is_7th_day',
       'Lights', 'Kitchen_T', 'Kitchen_H', 'Living_T', 'Living_H', 'Laundry_T',
       'Laundry_H', 'Office_T', 'Office_H', 'Bath_T', 'Bath_H', 'Outside_T',
       'Outside_H', 'Ironing_T', 'Ironing_H', 'TeenRoom_T', 'TeenRoom_H',
       'ParentsRoom_T', 'ParentsRoom_H', 'Weather_T', 'Weather_P', 'Weather_H',
       'Weather_WindSpeed', 'Weather_Visibility', 'Weather_Dewpoint', 'monday',
       'tuesday', 'wednesday', 'thursday', 'fri

In [12]:
X['Sin_day']

0        0.000000
1        0.043619
2        0.087156
3        0.130526
4        0.173648
           ...   
19730    0.087156
19731    0.130526
19732    0.173648
19733    0.216440
19734    0.258819
Name: Sin_day, Length: 19735, dtype: float64

In [5]:
def mape(y_test,y_pred):
    return np.mean(np.abs((y_test-y_pred)/y_test))

def rmse(y_test,y_pred):
    return np.mean((y_test-y_pred)**2)**0.5


def plot_model(model="Linear",l1_ratio=0.5,alpha=1):
    if model not in ["Linear","Elastic_net"]:
        raise ValueError("If have to choose either 'Linear' or 'Elastic_net'")
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    
    if model == "Linear":
        model1 = LinearRegression()
        model1.fit(X_train,y_train)

    elif model == "Elastic_net":
        model1 = ElasticNet(l1_ratio=l1_ratio,alpha=alpha)
        model1.fit(X_train,y_train)

    y_pred = borne_min(model1.predict(X_test),np.min(X_train))

    print(f"Le MAPE est {mape(y_test.to_numpy(),y_pred)}")
    print(f"Le RMSE est {rmse(y_test.to_numpy(),y_pred)}")

    if model == "Elastic_net":
        fig = go.Figure()

# Add the second line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_test, mode='lines', name='Test'))

# Add the first line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_pred, mode='lines', name='Prédiction'))

# Customize layout
        fig.update_layout(
        title=f"Prédiction {model}\n avec l1 = {l1_ratio} et alpha = {alpha}",
        xaxis_title="X-axis Label",
        yaxis_title="Y-axis Label",
        legend_title="Legend",
        template="plotly_white")

# Show the plot
        fig.show()


    else:
        fig = go.Figure()

# Add the second line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_test, mode='lines', name='Test'))

# Add the first line
        fig.add_trace(go.Scatter(x=data.index[:len(y_pred)], y=y_pred, mode='lines', name='Prédiction'))

# Customize layout
        fig.update_layout(
        title=f"Prédiction {model}",
        xaxis_title="X-axis Label",
        yaxis_title="Y-axis Label",
        legend_title="Legend",
        template="plotly_white")

# Show the plot
        fig.show()

        return y_pred    


In [6]:
def plot_online_model(model="Linear",l1_ratio=0.5,alpha=1,figure = True, arma_resid = False, order = (0,0,0), seasonnal_order = (0,0,0,0)):

    if model not in ["Linear","Elastic_net"]:
        raise ValueError("If have to choose either 'Linear' or 'Elastic_net'")

    step = 144 
    days = 1 
    total_steps = step * days
    
    
    y_pred = []
    
    for j in range(0, len(data) - train_index, total_steps):
        X_train, y_train = X.iloc[:train_index + j], y[:train_index + j]
        X_test = X.iloc[train_index + j:train_index + j + total_steps]
            
        model1 = ElasticNet(l1_ratio=l1_ratio, alpha=alpha) if model == "Elastic_net" else LinearRegression()
        model1.fit(X_train, y_train)

        y_train_pred = model1.predict(X_train)
        
        y_test_pred = model1.predict(X_test)

        if arma_resid:
            print('test')
            # Calculate residuals for the training set
            residuals = y_train - y_train_pred

            # Fit ARMA to the residuals
            arma_model = ARIMA(residuals, order=order, seasonal_order=seasonnal_order).fit()

            # Predict residuals for the test set
            residuals_pred = arma_model.forecast(steps=len(X_test))

            # Add predicted residuals to the main model's predictions
            y_test_pred += residuals_pred

        # Clip or adjust predictions as needed
        y_pred.append(borne_min(y_test_pred, np.min(y_train)))

    y_pred = np.hstack(y_pred)

    print(f"Le MAPE est {mape(y[train_index:train_index+len(y_pred)],y_pred)}")
    print(f"Le RMSE est {rmse(y_pred,y[train_index:train_index+len(y_pred)])}")

    if figure == True:
        fig = go.Figure()

# Add the second line
        fig.add_trace(go.Scatter(x=data.index[train_index:train_index+len(y_pred)], y=y[train_index:train_index+len(y_pred)], mode='lines', name='Test'))

# Add the first line
        fig.add_trace(go.Scatter(x=data.index[train_index:train_index+len(y_pred)], y=y_pred, mode='lines', name='Prédiction'))

# Customize layout
        fig.update_layout(
title=f"Prédiction {model}",
xaxis_title="X-axis Label",
yaxis_title="Y-axis Label",
legend_title="Legend",
template="plotly_white")

# Show the plot
        fig.show()  

    return y_pred

On cherche les contraintes l1 l2 optimale sur la base de données d'entrainements:

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

train_index = (data.index <= "2016-04-27 18:00:00").sum()
X_train, y_train = X.iloc[:train_index], y[:train_index]
# Elastic Net model
model = ElasticNet(random_state=42)

# Hyperparameter grid
param_distributions = {
    "alpha": np.logspace(-4, 1, 50),  # Regularization strength
    "l1_ratio": np.linspace(0, 1, 50),  # Mix of L1 and L2 regularization
}

# Define TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Randomized search
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=125,  # Number of random combinations
    scoring="neg_mean_squared_error",  # Metric
    cv=tscv,  # Sequential cross-validation
    random_state=42,
    n_jobs=-1
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

In [ ]:
from itertools import product

# Grille pour (p, d, q)
p = range(0, 4)  # Ordre AR
d = range(0, 2)  # Ordre de différenciation
q = range(0, 4)  # Ordre MA
order_grid = list(product(p, d, q))

P = range(0, 2)
D = range(0, 2)
Q = range(0, 2)
S = [144]  # Périodicité saisonnière (par exemple, mensuelle)

seasonal_order_grid = list(product(P, D, Q, S))

def optimize_aic_sarima(residuals, order_grid, seasonal_order_grid):
    best_order = None
    best_seasonal_order = None
    best_aic = float("inf")

    for order in order_grid:
        for seasonal_order in seasonal_order_grid:
            try:
                # Ajuster le modèle SARIMA
                sarima_model = ARIMA(residuals, order=order, seasonal_order=seasonal_order).fit()
                aic = sarima_model.aic
                if aic < best_aic:
                    best_aic = aic
                    best_order = order
                    best_seasonal_order = seasonal_order
            except Exception as e:
                continue

    return best_order, best_seasonal_order, best_aic

# Calculer les résidus Elastic Net
residuals = y_train - random_search.best_estimator_.predict(X_train)

# Optimiser l'AIC
best_order, best_aic = optimize_aic_sarima(residuals, order_grid, seasonal_order_grid)

print("Best ARMA Order:", best_order)
print("Best AIC:", best_aic)


In [ ]:
plot_online_model(model="Elastic_net",l1_ratio=0.9591836734693877,alpha=0.3727593720314942)

In [ ]:
plot_online_model(model="Elastic_net",l1_ratio=0.9591836734693877,alpha=0.3727593720314942,figure = True, arma_resid = True, order = (1,1,1), seasonnal_order = (1,0,1,14))